# Final Collocation with SEVIRI

This will be a 2 step process. Function 1 will be taking all of the advected files and removing all entries which are not integer multiples of 15 minutes, as we cannot make retrievals for these times anyway. These files are lifted from 
1. '/gws/nopw/j04/eo_shared_data_vol2/scratch/pete_nut/harry_advected/RealData/')
2. '/gws/nopw/j04/eo_shared_data_vol2/scratch/pete_nut/harry_advected/FittedData/')
3. '/gws/nopw/j04/eo_shared_data_vol2/scratch/pete_nut/harry_advected/UpperCounterFacData/')
4. '/gws/nopw/j04/eo_shared_data_vol2/scratch/pete_nut/harry_advected/LowerCounterFacData/')

with each being followed by 2019_{}_{}_{}:{}'.format(str(date.month).zfill(2), str(date.day).zfill(2), str(date.hour).zfill(2), str(date.minute).zfill(2))) 

And they are trimmed, and then returned to the locations at 
1. '/gws/nopw/j04/eo_shared_data_vol2/scratch/AO12/RealDataAdvected/'
2. '/gws/nopw/j04/eo_shared_data_vol2/scratch/AO12/FittedlDataAdvected/'
3. '/gws/nopw/j04/eo_shared_data_vol2/scratch/AO12/UpperCounterFacDataAdvected/'
4. '/gws/nopw/j04/eo_shared_data_vol2/scratch/AO12/LowerCounterFacDataAdvected/'
   

Function 1 below deals with this process. 

## Packages

In [15]:
import os 
import pathlib
from datetime import datetime, timedelta, time
import pickle
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from scipy.spatial import KDTree

## Function 1

In [2]:
# MissingFiles = []

# def TrimAndCopy(StartDate, EndDate, FileType):
#     Link = f'/gws/nopw/j04/eo_shared_data_vol2/scratch/pete_nut/harry_advected/{FileType}Data'
    
#     while StartDate < EndDate:
#         try:
#             L = Link + f"/{StartDate.year:04d}_{StartDate.month:02d}_{StartDate.day:02d}_{StartDate.hour:02d}:{StartDate.minute:02d}"
#             File1 = pd.read_csv(L)
            
#             # Filter based on latitude and longitude
#             File1 = File1[(File1['lat'] >= -22) & (File1['lat'] <= 0) & (File1['lon'] >= -16) & (File1['lon'] <= 10)]
            
#             File1['jday'] = pd.to_datetime(File1['jday'], format='%Y-%m-%d %H:%M:%S', errors='coerce')
            
#             PlaceholderName = File1[File1['jday'].dt.minute % 15 == 0]
            
#             PlaceholderName.to_csv('/gws/nopw/j04/eo_shared_data_vol2/scratch/AO12/{}DataAdvected/{}_{}_{}_{}:{}'.format(
#                 FileType, str(StartDate.year), str(StartDate.month).zfill(2), str(StartDate.day).zfill(2),
#                 str(StartDate.hour).zfill(2), str(StartDate.minute).zfill(2)))
            
#             print(f"Date: {StartDate}, Link = {'/gws/nopw/j04/eo_shared_data_vol2/scratch/AO12/{}DataAdvected/{}_{}_{}_{}:{}'.format(FileType, str(StartDate.year), str(StartDate.month).zfill(2), str(StartDate.day).zfill(2),str(StartDate.hour).zfill(2), str(StartDate.minute).zfill(2))}, Complete")
            
#             StartDate += timedelta(hours=1)
#         except FileNotFoundError:
#             MissingFiles.append(f'{StartDate}')
#             print(f'File for {StartDate} is missing')
#             StartDate += timedelta(hours=1)

#Heres what this is doing in words: take any given file from the Link, and it is made up of a real measurement for each particle, 
#which is then advected every 5 minutes. As SEVIRI only has 15 minute resolution, I take these copies and I move them to another location, and along the
# way i get rid of every time measurement that will not end up being relevant, and all long- lat pairings which are not useful to our experiment. 

In [3]:
# TrimAndCopy(StartDate = datetime(2019, 1, 1, 0, 0), EndDate = datetime(2019, 2, 1, 0, 0), FileType = 'Real')
# FileType = ['Real', 'Fitted', 'UpperCounterFac', 'LowerCounterFac']

In [22]:
# file = pd.read_csv('/gws/nopw/j04/eo_shared_data_vol2/scratch/AO12/RealDataAdvected/2019_01_01_00:00')                  
# file

## Function 2

In [2]:
def CollocationFunction(StartDate, EndDate, FileType):
    while StartDate < EndDate:
        d = StartDate
        print(d)
        try:
            TrimmedAdvectedEmissions = pd.read_csv(f'/gws/nopw/j04/eo_shared_data_vol2/scratch/AO12/{FileType}DataAdvected/{StartDate.year:04d}_{StartDate.month:02d}_{StartDate.day:02d}_{StartDate.hour:02d}:{StartDate.minute:02d}') 
            TrimmedAdvectedEmissions['jday'] = pd.to_datetime(TrimmedAdvectedEmissions['jday'])
            if 'cer' in TrimmedAdvectedEmissions.columns:
                print(f'Data already collocated for {StartDate}')
                StartDate += timedelta(hours = 1)
                continue
            grouped_data = TrimmedAdvectedEmissions.groupby('jday')
            subsets = [group for _, group in grouped_data]
            new_groups = []
            
            while d < StartDate + timedelta(hours = 24, minutes = 1): # Increment date here by 1 in hour column
                try:
                    newPath = f'/gws/nopw/j04/eo_shared_data_vol1/satellite/seviri-orac/Data/{d.year:04d}/{d.month:02d}/{d.day:02d}/{d.hour:02d}/{d.year:04d}{d.month:02d}{d.day:02d}{d.hour:02d}{d.minute:02d}{d.second:02d}-ESACCI-L2_CLOUD-CLD_PRODUCTS-SEVIRI-MSG4-fv3.0.nc'
                    cloud_ds = xr.open_dataset(newPath, decode_times=False)
                
                    wh_seviri_finite = np.isfinite(cloud_ds.lon.data.ravel())
                    seviri_lonlat_tree = KDTree(np.stack([cloud_ds.lon.data.ravel()[wh_seviri_finite], cloud_ds.lat.data.ravel()[wh_seviri_finite]], -1))
                    
                    modified_fitted_group = None
                    
                    for group in subsets:
                        time_difference = abs(group['jday'].iloc[0] - d)
                        if time_difference < timedelta(minutes = 5): 
                            wh_emissions_finite = np.isfinite(group.lon.to_numpy())
                            dists, nearest_neighbours = seviri_lonlat_tree.query(np.stack([
                                group.lon.to_numpy()[wh_emissions_finite], 
                                group.lat.to_numpy()[wh_emissions_finite]
                            ], -1))
                          
                            wh_emissions_valid = wh_emissions_finite[~np.isnan(wh_emissions_finite)]
                            
                            # Retrieve all desired variables from the cloud_ds data frame
                            extracted_values1  = cloud_ds.lat.data.ravel()[wh_seviri_finite][nearest_neighbours]
                            extracted_values2  = cloud_ds.lon.data.ravel()[wh_seviri_finite][nearest_neighbours] 
                            extracted_values3  = cloud_ds.cot.data.ravel()[wh_seviri_finite][nearest_neighbours]
                            extracted_values4  = cloud_ds.cot_uncertainty.data.ravel()[wh_seviri_finite][nearest_neighbours]                   
                            extracted_values5  = cloud_ds.cer.data.ravel()[wh_seviri_finite][nearest_neighbours]
                            extracted_values6  = cloud_ds.cer_uncertainty.data.ravel()[wh_seviri_finite][nearest_neighbours]                    
                            extracted_values7  = cloud_ds.cwp.data.ravel()[wh_seviri_finite][nearest_neighbours]
                            extracted_values8  = cloud_ds.cwp_uncertainty.data.ravel()[wh_seviri_finite][nearest_neighbours]
                            extracted_values9  = cloud_ds.cth.data.ravel()[wh_seviri_finite][nearest_neighbours]
                            extracted_values10 = cloud_ds.cth_uncertainty.data.ravel()[wh_seviri_finite][nearest_neighbours]
                            extracted_values11 = cloud_ds.illum.data.ravel()[wh_seviri_finite][nearest_neighbours]
        
                            # Update a copy of the group
                            modified_fitted_group = group.copy()
                            modified_fitted_group['pixel lat'] = np.nan                
                            modified_fitted_group['pixel lon'] = np.nan
                            modified_fitted_group['cot'] = np.nan
                            modified_fitted_group['cot_uncertainty'] = np.nan                
                            modified_fitted_group['cer'] = np.nan
                            modified_fitted_group['cer_uncertainty'] = np.nan
                            modified_fitted_group['cwp'] = np.nan                
                            modified_fitted_group['cwp_uncertainty'] = np.nan
                            modified_fitted_group['cth'] = np.nan
                            modified_fitted_group['cth_uncertainty'] = np.nan                
                            modified_fitted_group['illum'] = np.nan
        
                            
            
                            modified_fitted_group.loc[wh_emissions_valid, 'pixel lat'] = extracted_values1
                            modified_fitted_group.loc[wh_emissions_valid, 'pixel lon'] = extracted_values2                
                            modified_fitted_group.loc[wh_emissions_valid, 'cot'] = extracted_values3
                            modified_fitted_group.loc[wh_emissions_valid, 'cot_uncertainty'] = extracted_values4
                            modified_fitted_group.loc[wh_emissions_valid, 'cer'] = extracted_values5
                            modified_fitted_group.loc[wh_emissions_valid, 'cer_uncertainty'] = extracted_values6
                            modified_fitted_group.loc[wh_emissions_valid, 'cwp'] = extracted_values7
                            modified_fitted_group.loc[wh_emissions_valid, 'cwp_uncertainty'] = extracted_values8
                            modified_fitted_group.loc[wh_emissions_valid, 'cth'] = extracted_values9
                            modified_fitted_group.loc[wh_emissions_valid, 'cth_uncertainty'] = extracted_values10
                            modified_fitted_group.loc[wh_emissions_valid, 'illum'] = extracted_values11
            
                            
                            # Append the modified group to the new list
                            new_groups.append(modified_fitted_group)
                            
                    print(d)
                    d += timedelta(minutes=15)
                    
                except FileNotFoundError:
                   # print('no data available for ' + str(d))
                    # Create a new group with NaN values
                    new_group = pd.DataFrame(columns=TrimmedAdvectedEmissions.columns)
                    new_group['jday'] = [d] * len(new_group)
                    new_group['cot'] = np.nan
                    new_group['cot_uncertainty'] = np.nan
                    new_groups.append(new_group)
                    d += timedelta(minutes=15)
        except FileNotFoundError:
            print(f'No data Available to collocate with for {StartDate}')
        
        # Concatenate the modified groups into a new DataFrame
        new_trimmed_advected_emissions = pd.concat(new_groups, ignore_index=True)
        #print(new_trimmed_advected_emissions)
        new_trimmed_advected_emissions.to_csv('/gws/nopw/j04/eo_shared_data_vol2/scratch/AO12/{}DataAdvected/{}_{}_{}_{}:{}'.format(FileType, str(StartDate.year), str(StartDate.month).zfill(2), str(StartDate.day).zfill(2), str(StartDate.hour).zfill(2), str(StartDate.minute).zfill(2))) 
        print(f'Data for {StartDate} added')
        StartDate += timedelta(hours = 1)


In [3]:
# CollocationFunction(StartDate = datetime(2019, 1, 1, 0, 0), EndDate = datetime(2019, 1, 1, 3, 1), FileType = 'Real')

In [ ]:
import multiprocessing
start_dates = []
end_dates   = []
file_types  = []
if __name__ == '__main__':
    # Define your input parameters
    
    for integer in range(0, 30):
        start_dates.append(datetime(2019, 1, 1, 0, 0) + integer * timedelta(days = 1) ) 
        end_dates.append(datetime(2019, 1, 1, 6, 1)  + integer * timedelta(days = 1) )
        file_types.append('Real')

    # Combine the input parameters into a list of tuples
    input_parameters = list(zip(start_dates, end_dates, file_types))

    with multiprocessing.Pool(processes=10) as pool:
        # Use starmap to pass multiple arguments to the function
        results = pool.starmap(CollocationFunction, input_parameters)

## misc

In [5]:
# file = os.listdir('/gws/nopw/j04/eo_shared_data_vol2/scratch/AO12/RealDataAdvected/')  # STARTED AT 11:50
# for item in file:
#     print(item)
# item = pd.read_csv('/gws/nopw/j04/eo_shared_data_vol2/scratch/AO12/RealDataAdvected/2019_01_01_00:00')

In [21]:
# file = pd.read_csv('/gws/nopw/j04/eo_shared_data_vol2/scratch/AO12/RealDataAdvected/2019_01_01_00:00')
# file.sort_values(by = 'cot' )


In [ ]:
# files = pd.read_csv('/gws/nopw/j04/eo_shared_data_vol2/scratch/pete_nut/harry_advected/RealData/2019_01_18_13:00')
# files

In [ ]:
# from datetime import datetime, timedelta
# import os
# List = []
# StartDate = datetime(2019, 1, 1, 0, 0)

# while StartDate < datetime(2019, 2, 1, 0, 1):
#     try:
#         file_path = '/gws/nopw/j04/eo_shared_data_vol2/scratch/AO12/RealDataAdvected/{}_{}_{}_{}:{}'.format(
#             str(StartDate.year), str(StartDate.month).zfill(2), str(StartDate.day).zfill(2),
#             str(StartDate.hour).zfill(2), str(StartDate.minute).zfill(2)
#         )
        
#         # Check if file exists and has a non-zero size
#         if os.path.exists(file_path) and os.path.getsize(file_path) > 0:
#             print(f'The date {StartDate} has a valid file at {file_path} with size {os.path.getsize(file_path)} bytes')
#             if os.path.getsize(file_path) < 1000:
#                 List.append(str(StartDate))
#         else:
#             print(f'The date {StartDate} has no valid file or has zero size')
#     except FileNotFoundError:
#         print('n')
#     StartDate += timedelta(hours=1)
# print(List)

## Progress checks on nohup code

In [20]:
Done  = 0
Total = 0 
Missing = 0
StartDate = datetime(2019, 1, 1, 0, 0)
while StartDate < datetime(2019, 2, 1, 0, 1):
    try:
        TrimmedAdvectedEmissions = pd.read_csv(f'/gws/nopw/j04/eo_shared_data_vol2/scratch/AO12/RealDataAdvected/{StartDate.year:04d}_{StartDate.month:02d}_{StartDate.day:02d}_{StartDate.hour:02d}:{StartDate.minute:02d}') 
        if 'cer' in TrimmedAdvectedEmissions.columns:
            Done  += 1
            Total += 1
            StartDate += timedelta(hours = 1)
        else:
            Total += 1
            StartDate += timedelta(hours = 1)
    except FileNotFoundError:
        Missing += 1
        StartDate += timedelta(hours = 1)
print(f'As of {datetime.now()}, we have completed {Done} out of {Total} files, and there are {Missing} missing files in January')
            

As of 2024-01-30 13:38:39.377245, we have completed 232 out of 653 files, and there are 92 missing files in January
